In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd

In [2]:
class ClassifyNet(nn.Module):
    def __init__(self, num_in, num_hidden_1, num_hidden_2, num_out):
        super(ClassifyNet, self).__init__()
        self.fc1 = nn.Linear(num_in,num_hidden_1)
        self.dropout1 = nn.Dropout(0.33)
        self.fc2 = nn.Linear(num_hidden_1,num_hidden_2)
        self.dropout2 = nn.Dropout(0.33)
        self.fc3 = nn.Linear(num_hidden_2, num_out)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output

In [3]:
num_hidden_1 = 1024
num_hidden_2 = 512
batch_size = 1000

#GPU
cuda = True
use_cuda = cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print('Using ',device)

#Load Data from CSV and train test split
dataset = pd.read_csv('CleanedData _pytorch.csv')
dataset = np.array(dataset)

data = dataset[:, :-1]
target = dataset[:, -1]

#Training data (select at random from first 50000)
data_size = len(data) - 1
data_ind = np.random.permutation(data_size)[:data_size]

#Convert data to torch and device
data_train = torch.from_numpy(data[data_ind[:50000]]).float().to(device)
target_train = torch.from_numpy(target[data_ind[:50000]]).long().to(device)
data_test = torch.from_numpy(data[data_ind[50000:]]).float().to(device)
target_test = torch.from_numpy(target[data_ind[50000:]]).long().to(device)

Using  cuda


In [4]:
#Setup model and optimizer
model = ClassifyNet(42,num_hidden_1,num_hidden_2,3).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.005)  #Learning rates

In [5]:
#Training
print('Iteration,Testing Accuracy,Training Accuracy')
for i in range(250):

    #Model evaluation
    model.eval()
    with torch.no_grad():
        pred = torch.argmax(model(data_test),axis=1)
        test_accuracy = torch.sum(pred == target_test)/len(pred)
        pred = torch.argmax(model(data_train),axis=1)
        train_accuracy = torch.sum(pred == target_train)/len(pred)
        print(i,test_accuracy.item()*100,train_accuracy.item()*100)

    #Training mode, run data through neural network in mini-batches (SGD)
    model.train()
    for j in range(0,len(target_train),batch_size):
        optimizer.zero_grad()
        loss = F.nll_loss(model(data_train[j:j+batch_size,:]), target_train[j:j+batch_size])
        loss.backward()
        optimizer.step()

Iteration,Testing Accuracy,Training Accuracy
0 12.5213623046875 12.603999674320221
1 67.64270067214966 67.57400035858154
2 73.65273237228394 73.89799952507019
3 77.52079367637634 78.1819999217987
4 78.53480577468872 79.02799844741821
5 79.11587357521057 80.07000088691711
6 80.06722331047058 81.44399523735046
7 80.33496737480164 81.65199756622314
8 80.61410784721375 82.03199505805969
9 81.13250732421875 82.88799524307251
10 81.26922845840454 82.98799991607666
11 81.34897947311401 83.46399664878845
12 81.65660500526428 83.91799926757812
13 81.75914287567139 84.1539978981018
14 82.03258514404297 84.41599607467651
15 81.97562098503113 84.83999967575073
16 82.21487998962402 84.95999574661255
17 82.11803436279297 85.25199890136719
18 82.18070268630981 85.56599617004395
19 82.18070268630981 85.70199608802795
20 82.20348954200745 85.61399579048157
21 82.32311606407166 85.90399622917175
22 82.61365294456482 86.2779974937439
23 82.92127251625061 86.63599491119385
24 82.4541449546814 86.845999956

In [6]:
torch.argmax(model(data_test[:20]),axis=1)

tensor([0, 0, 1, 0, 1, 0, 0, 2, 0, 0, 1, 1, 1, 0, 0, 0, 0, 2, 0, 0],
       device='cuda:0')

In [7]:
target_test[:20]

tensor([0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0],
       device='cuda:0')